In [144]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
import numpy as np
import re
import streamlit as st 
import plotly.graph_objects as go


In [17]:
ISH = pd.read_csv('../CleanData/ISH_summary.csv', index_col=0)
BOM = pd.read_csv('../CleanData/BOM_summary.csv', index_col=0)
cr_chapter = pd.read_csv('../CleanData/by_chapter_cross_ref.csv', index_col=0)
cr_verse = pd.read_csv('../CleanData/by_verse_cross_ref.csv', index_col=0)

In [261]:
# Data Prep 
ISH_verse_df = cr_verse.dropna(subset=['verse_title_ISH'])
BOM_verse_df = cr_verse.dropna(subset=['verse_title_BOM'])
# Verse comparison

select_ISH_verse = "Isaiah 1:1"
select_BOM_verse = '2 Nephi 6:4'


In [265]:
verse_tb = cr_verse[(cr_verse['verse_title_ISH'] == select_ISH_verse) & (cr_verse['verse_title_BOM'] == select_BOM_verse)][['scripture_text_ISH', 'scripture_text_BOM', 'similarity_score', 'similarity_category', 'Duhms_Class']]
verse_tb.columns = [select_ISH_verse, select_BOM_verse, 'Similarity Score', 'Similarity Category', "Duhm's Category"]

verse_tb['Similarity Score'] = round(verse_tb['Similarity Score'], ndigits=4)
verse_tb['Similarity Score']


1    0.1033
Name: Similarity Score, dtype: float64

In [148]:
# Select Isaiah Chapter
select_chapter_number = "Isaiah 2"
ch_num = int(re.search(r'\d+', select_chapter_number).group())

# Create bar data
ISH_ch = ISH[ISH['combo_ISH'] == select_chapter_number].rename({'bible_term_in_ISH': "Bible Term", 'verse_count_ISH': "Verse Count"}, axis=1).melt(
    id_vars=['combo_ISH', 'chapter_number_ISH'],
    value_vars=['Direct Quote', 'Shared Language', 'Similar Theme', 'Bible Term', 'Verse Count'],
    var_name='Type', value_name='Count'
)

# custom palatte mapping from SET2 in seaborn
pal = sns.color_palette('Set2').as_hex()
colors = [pal[i] for i in range(5)]

color_mapping={
        'Direct Quote': colors[0],
        'Shared Language': colors[1],
        'Similar Theme': colors[2],
        'Bible Term': colors[3],
        'Verse Count': colors[4]
    }

# Generate bar data
bar = px.bar(data_frame=ISH_ch, x='Type', y='Count', color = 'Type', title=f"Summary of {select_chapter_number}", color_discrete_map=color_mapping)

hovertemplate = '<b>Count:</b> %{y}<b>'

bar.update_traces(
    hovertemplate=hovertemplate,
)

bar.show()





In [239]:

select_book = '1 Nephi'

# Select Isaiah Chapter
if select_book == "Isaiah":
    select_chapter_number = "Isaiah 2"

    # Create bar data
    ch_data = ISH[ISH['combo_ISH'] == select_chapter_number].rename({'bible_term_in_ISH': "Bible Term", 'verse_count_ISH': "Verse Count"}, axis=1).melt(
        id_vars=['combo_ISH', 'chapter_number_ISH'],
        value_vars=['Direct Quote', 'Shared Language', 'Similar Theme', 'Bible Term', 'Verse Count'],
        var_name='Type', value_name='Count')

# Select BOM Chapter
else: 
    select_chapter_number = '1 Nephi 18'

    # Create bar data
    ch_data = BOM[BOM['combo_BOM'] == select_chapter_number].rename({'bible_term_in_BOM': "Bible Term", 'verse_count_BOM': "Verse Count"}, axis=1).melt(
        id_vars=['combo_BOM', 'chapter_number_BOM'],
        value_vars=['Direct Quote', 'Shared Language', 'Similar Theme', 'Bible Term', 'Verse Count'],
        var_name='Type', value_name='Count')

ch_num = int(re.findall(r'\d+', select_chapter_number)[-1])

# custom palatte mapping from SET2 in seaborn
pal = sns.color_palette('Set2').as_hex()
colors = [pal[i] for i in range(5)]

color_mapping={
        'Direct Quote': colors[0],
        'Shared Language': colors[1],
        'Similar Theme': colors[2],
        'Bible Term': colors[3],
        'Verse Count': colors[4]
    }

# Generate bar data
bar = px.bar(data_frame=ch_data, x='Type', y='Count', color = 'Type', title=f"Summary of {select_chapter_number}", color_discrete_map=color_mapping)

hovertemplate = '<b>Count:</b> %{y}<b>'
bar.update_layout(xaxis_title=None, legend_title = None)

bar.update_traces(
    hovertemplate=hovertemplate,
)

bar

In [220]:
# table data
if select_book == "Isaiah":
    cr_ch = cr_verse[cr_verse['chapter_number_ISH'] == ch_num]
    DQ = cr_ch[cr_ch['similarity_category'] == "Direct Quote"]['verse_title_BOM'].reset_index(drop=True)
    SL = cr_ch[cr_ch['similarity_category'] == "Shared Language"]['verse_title_BOM'].reset_index(drop=True)
    ST = cr_ch[cr_ch['similarity_category'] == "Similar Theme"]['verse_title_BOM'].reset_index(drop=True)
    BT = cr_ch[cr_ch['bible_term_in_ISH'] == True].sort_values('verse_number_ISH')['verse_title_ISH'].reset_index(drop=True).drop_duplicates()

    ch_tb = pd.concat([DQ, SL, ST, BT], axis=1).reset_index(drop=True).fillna('-')
    ch_tb.columns = ['Direct Quote', 'Shared Language', 'Similar Theme', 'Bible Term Verses']


else:
    cr_ch = cr_verse[(cr_verse['chapter_number_BOM'] == ch_num) & (cr_verse['book_title_BOM'] == select_book)]
    DQ = cr_ch[cr_ch['similarity_category'] == "Direct Quote"]['verse_title_ISH'].reset_index(drop=True)
    SL = cr_ch[cr_ch['similarity_category'] == "Shared Language"]['verse_title_ISH'].reset_index(drop=True)
    ST = cr_ch[cr_ch['similarity_category'] == "Similar Theme"]['verse_title_ISH'].reset_index(drop=True)
    BT = cr_ch[cr_ch['bible_term_in_BOM'] == True].sort_values('verse_number_BOM')['verse_title_BOM'].reset_index(drop=True).drop_duplicates()

    ch_tb = pd.concat([DQ, SL, ST, BT], axis=1).reset_index(drop=True).fillna('-')
    ch_tb.columns = ['Direct Quote', 'Shared Language', 'Similar Theme', 'Bible Term Verses']


In [271]:
# st.markdown(ch_tb.style.hide(axis="index").to_html(), unsafe_allow_html=True)
cr_verse

,verse_title_ISH,book_title_ISH,chapter_number_ISH,verse_number_ISH,scripture_text_ISH,verse_title_BOM,book_title_BOM,chapter_number_BOM,verse_number_BOM,scripture_text_BOM,scripture_cleaned_ISH,scripture_cleaned_BOM,similarity_score,similarity_category,word_count_ISH,word_count_BOM,bible_term_in_ISH,bible_term_in_BOM,Duhms_Class
0,Isaiah 1:1,Isaiah,1.0,1.0,"THE vision of Isaiah the son of Amoz, which he...",1 Nephi 19:23,1 Nephi,19.0,23.0,And I did read many things unto them which wer...,vision isaiah son amoz saw concerning judah je...,read many things unto written books moses migh...,0.051640,Similar Theme,27,61,True,True,Proto
1,Isaiah 1:1,Isaiah,1.0,1.0,"THE vision of Isaiah the son of Amoz, which he...",2 Nephi 6:4,2 Nephi,6.0,4.0,"And now, behold, I would speak unto you concer...",vision isaiah son amoz saw concerning judah je...,behold would speak unto concerning things come...,0.103280,Similar Theme,27,61,True,True,Proto
2,Isaiah 1:1,Isaiah,1.0,1.0,"THE vision of Isaiah the son of Amoz, which he...",2 Nephi 11:8,2 Nephi,11.0,8.0,"And now I write some of the words of Isaiah, t...",vision isaiah son amoz saw concerning judah je...,write words isaiah whoso people shall see word...,0.057735,Similar Theme,27,45,True,True,Proto
3,Isaiah 1:1,Isaiah,1.0,1.0,"THE vision of Isaiah the son of Amoz, which he...",2 Nephi 25:1,2 Nephi,25.0,1.0,"Now I, Nephi, do speak somewhat concerning the...",vision isaiah son amoz saw concerning judah je...,nephi speak somewhat concerning words written ...,0.105409,Similar Theme,27,50,True,True,Proto
4,Isaiah 1:1,Isaiah,1.0,1.0,"THE vision of Isaiah the son of Amoz, which he...",3 Nephi 20:11,3 Nephi,20.0,11.0,"Ye remember that I spake unto you, and said th...",vision isaiah son amoz saw concerning judah je...,ye remember spake unto said words isaiah fulfi...,0.071611,Similar Theme,27,29,True,True,Proto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,NaN,NaN,NaN,NaN,NaN,Moroni 10:29,Moroni,10.0,29.0,"And God shall show unto you, that that which I...",NaN,god shall show unto written true,0.000000,NaN,0,14,False,True,NaN
7476,NaN,NaN,NaN,NaN,NaN,Moroni 10:30,Moroni,10.0,30.0,And again I would exhort you that ye would com...,NaN,would exhort ye would come unto christ lay hol...,0.000000,NaN,0,29,False,True,NaN
7477,NaN,NaN,NaN,NaN,NaN,Moroni 10:32,Moroni,10.0,32.0,"Yea, come unto Christ, and be perfected in him...",NaN,yea come unto christ perfected deny ungodlines...,0.000000,NaN,0,72,False,True,NaN
7478,NaN,NaN,NaN,NaN,NaN,Moroni 10:33,Moroni,10.0,33.0,"And again, if ye by the grace of God are perfe...",NaN,ye grace god perfect christ deny power ye sanc...,0.000000,NaN,0,57,False,True,NaN
